In [222]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
import pickle as pickle
# 이거 데이터프레임 숫자 표시 개수 정해주는 거
pd.options.display.float_format = '{:.4f}'.format
import warnings
warnings.filterwarnings('ignore')
import matplotlib.gridspec as gridspec

In [223]:
dev_loss_st = pd.read_csv('/opt/ml/code/prediction/0_dev_loss_st.csv')
train = pd.read_csv('/opt/ml/dataset/train/train.csv')
test = pd.read_csv('/opt/ml/code/prediction/0_test_st.csv')

In [3]:
test=test.drop(columns='gold_label')

In [4]:
test['probs'].apply(lambda x: str(np.append([1],np.zeros_like(eval(test.iloc[0,2]))[:-1])))

0       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
1       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
2       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
3       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
4       [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
                              ...                        
7760    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7761    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7762    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7763    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
7764    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
Name: probs, Length: 7765, dtype: object

In [5]:
test['probs'].apply(lambda x: str([0]*30))

0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                              ...                        
7760    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7761    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7762    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7763    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7764    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: probs, Length: 7765, dtype: object

In [6]:
# test.to_csv('/opt/ml/code/prediction/24 new_meta_binary2.csv',index=False)

In [7]:
num_label = []
label = dev_loss_st['gold_label'].unique()
with open('/opt/ml/code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

In [8]:
pd.DataFrame([dict_label_to_num])

,no_relation,org:top_members/employees,org:members,org:product,per:title,org:alternate_names,per:employee_of,org:place_of_headquarters,per:product,org:number_of_employees/members,...,org:member_of,per:parents,org:dissolved,per:schools_attended,per:date_of_death,per:date_of_birth,per:place_of_birth,per:place_of_death,org:founded_by,per:religion
0,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29


In [23]:
def train_pred(train):
    subjt=train['subject_entity'].apply(lambda x: pd.Series(eval(x))).add_prefix('subj_')
    objt=train['object_entity'].apply(lambda x: pd.Series(eval(x))).add_prefix('obj_')
    full_train = pd.concat([train[['id','sentence','source','label']],subjt,objt],axis=1)
    del subjt,objt
    return full_train

In [24]:
full_train=train_pred(train)

In [25]:
train.head(5)

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [26]:
full_train.head(5)

,id,sentence,source,label,subj_word,subj_start_idx,subj_end_idx,subj_type,obj_word,obj_start_idx,obj_end_idx,obj_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,wikipedia,no_relation,비틀즈,24,26,ORG,조지 해리슨,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,wikitree,no_relation,민주평화당,19,23,ORG,대안신당,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,wikitree,org:member_of,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,wikitree,org:top_members/employees,아성다이소,13,17,ORG,박정부,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,wikipedia,no_relation,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT


In [27]:
full_train[full_train['subj_type']=='PER']['obj_type'].unique()

array(['DAT', 'ORG', 'LOC', 'POH', 'PER', 'NOH'], dtype=object)

In [28]:
full_train[full_train['subj_type']=='ORG']['obj_type'].unique()

array(['PER', 'ORG', 'DAT', 'LOC', 'POH', 'NOH'], dtype=object)

In [30]:
full_train[(full_train['subj_type']=='PER')&(full_train['subj_type']=='PER')]['label'].unique()

array(['per:date_of_birth', 'per:employee_of', 'per:origin', 'per:title',
       'no_relation', 'per:schools_attended', 'per:colleagues',
       'per:alternate_names', 'per:spouse', 'per:children',
       'per:place_of_birth', 'per:parents', 'per:religion',
       'per:date_of_death', 'per:place_of_residence', 'per:other_family',
       'per:siblings', 'per:product', 'per:place_of_death', 'org:members',
       'org:top_members/employees', 'org:founded_by',
       'org:place_of_headquarters'], dtype=object)

In [33]:
full_train['bi_label'] = full_train['label'].apply(lambda x: x=='no_relation' if x, 'relation'))

AttributeError: 'str' object has no attribute 'where'

In [ ]:
multi -> per : per / label 5개 아니면 no relation

In [4]:
a=train[train['subject_entity'].apply(lambda x: eval(x)['type']=='PER')]

In [7]:
b = train[train['object_entity'].apply(lambda x: eval(x)['type']=='PER')]

In [12]:
c = pd.merge(a,b,how='inner')

In [14]:
c

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [15]:
c[c['object_entity'].apply(lambda x: eval(x)['type']=='PER')]

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [16]:
c

,id,sentence,subject_entity,object_entity,label,source
0,20,1971년 대선을 앞두고 김종필은 1971년 선거에서 박정희 당선을 위해 무려 60...,"{'word': '김종필', 'start_idx': 14, 'end_idx': 16...","{'word': '박정희', 'start_idx': 30, 'end_idx': 32...",per:colleagues,wikipedia
1,21,2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 ...,"{'word': '스티브 바라캇', 'start_idx': 67, 'end_idx'...","{'word': 'Steve Barakatt', 'start_idx': 75, 'e...",per:alternate_names,wikipedia
2,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",per:spouse,wikipedia
3,25,"SK는 또 ""해당 방송에서 언급한 지난해 12월5일 가로세로연구소 유튜브 방송 또한...","{'word': '노소영', 'start_idx': 110, 'end_idx': 1...","{'word': '최태원', 'start_idx': 67, 'end_idx': 69...",per:spouse,wikitree
4,27,"1824년에 있었던 첫 졸업식에는 대통령 제임스 먼로와 존 C. 칼훈, 헨리 클레이...","{'word': '존 C. 칼훈', 'start_idx': 31, 'end_idx'...","{'word': '제임스 먼로', 'start_idx': 23, 'end_idx':...",no_relation,wikipedia
...,...,...,...,...,...,...
5004,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...","{'word': '에다노 유키오', 'start_idx': 25, 'end_idx'...","{'word': '센고쿠 요시토', 'start_idx': 42, 'end_idx'...",no_relation,wikipedia
5005,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...",per:parents,wikipedia
5006,32454,신라에서는 중기까지도 지배계층의 가계계승에 있어서 모계적인 경향이 그대로 존속되었고...,"{'word': '선덕여왕', 'start_idx': 101, 'end_idx': ...","{'word': '진덕여왕', 'start_idx': 95, 'end_idx': 9...",no_relation,wikipedia
5007,32463,"그의 외삼촌은 루이 13세로, 남편인 루이 14세는 외사촌 오빠가 된다.","{'word': '루이 14세', 'start_idx': 21, 'end_idx':...","{'word': '루이 13세', 'start_idx': 8, 'end_idx': ...",per:other_family,wikipedia


In [17]:
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [19]:
train[train['label']!='no_relation']

,id,sentence,subject_entity,object_entity,label,source
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
5,5,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...",org:alternate_names,wikipedia
7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...",per:date_of_birth,wikipedia
8,8,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 ...,"{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...","{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...",org:place_of_headquarters,wikipedia
...,...,...,...,...,...,...
32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '이창진', 'start_idx': 12, 'end_idx': 14...","{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...",per:title,wikitree
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree


In [33]:
t=np.zeros(30)

In [37]:
TEST_LABEL_TO_ID = {'no_relation':0, 'org:top_members/employees':1, 'org:members':2, 'org:product':3, 
'per:title':4,}

In [38]:
TEST_LABEL_TO_ID['no_relation']

0

In [39]:
zeros = np.zeros(30)

In [40]:
label_idx = [1, 3, 5, 7]

In [86]:
test.loc[test['id'] == 0, 'pred_label'] = -500

In [87]:
test

,id,gold_label,pred_label,probs,zeros
0,0,100,-500,"[0.0, 0.00057884311536327, 0.0, 0.000218044515...",0
1,1,100,per:alternate_names,"[0.0, 3.022115151907201e-06, 0.0, 4.5307548134...",0
2,2,100,no_relation,"[0.0, 0.9998899698257446, 0.0, 7.7153172242105...",0
3,3,100,per:other_family,"[0.0, 0.41120561957359314, 0.0, 0.000210893806...",0
4,4,100,org:top_members/employees,"[0.0, 0.0012245755642652512, 0.0, 0.9984153509...",0
...,...,...,...,...,...
7760,7760,100,no_relation,"[0.0, 0.6559950709342957, 0.0, 0.0007126444834...",0
7761,7761,100,no_relation,"[0.0, 0.999886155128479, 0.0, 4.99390716868219...",0
7762,7762,100,org:top_members/employees,"[0.0, 1.0821405339811463e-05, 0.0, 0.999811589...",0
7763,7763,100,no_relation,"[0.0, 0.9923446774482727, 0.0, 0.0001384720671...",0


In [85]:
test[test['id']==0]['pred_label']

0    org:members
Name: pred_label, dtype: object

In [84]:
test

,id,gold_label,pred_label,probs,zeros
0,0,100,org:members,"[0.0, 0.00057884311536327, 0.0, 0.000218044515...",0
1,1,100,per:alternate_names,"[0.0, 3.022115151907201e-06, 0.0, 4.5307548134...",0
2,2,100,no_relation,"[0.0, 0.9998899698257446, 0.0, 7.7153172242105...",0
3,3,100,per:other_family,"[0.0, 0.41120561957359314, 0.0, 0.000210893806...",0
4,4,100,org:top_members/employees,"[0.0, 0.0012245755642652512, 0.0, 0.9984153509...",0
...,...,...,...,...,...
7760,7760,100,no_relation,"[0.0, 0.6559950709342957, 0.0, 0.0007126444834...",0
7761,7761,100,no_relation,"[0.0, 0.999886155128479, 0.0, 4.99390716868219...",0
7762,7762,100,org:top_members/employees,"[0.0, 1.0821405339811463e-05, 0.0, 0.999811589...",0
7763,7763,100,no_relation,"[0.0, 0.9923446774482727, 0.0, 0.0001384720671...",0


In [60]:
test['zeros']

0       0
1       0
2       0
3       0
4       0
       ..
7760    0
7761    0
7762    0
7763    0
7764    0
Name: zeros, Length: 7765, dtype: int64

In [67]:
eval(test['probs'].iloc[0])[1] = 0.3333

In [70]:
eval(test['probs'].iloc[0])

[0.00057884311536327,
 0.0002180445153499022,
 0.9639279246330261,
 0.003564955433830619,
 7.976425695233047e-05,
 0.02826537750661373,
 1.8083310351357795e-05,
 0.00017945178842637688,
 2.440638309053611e-05,
 0.00044030899880453944,
 0.0001309694634983316,
 5.976915781502612e-05,
 6.130493420641869e-05,
 8.238356531364843e-05,
 4.882526627625339e-05,
 4.01366050937213e-05,
 4.160825483268127e-05,
 3.313729393994436e-05,
 2.060331462416798e-05,
 0.0012245550751686096,
 0.00017544497677590698,
 7.830125832697377e-05,
 4.6499018935719505e-05,
 7.592483598273247e-05,
 6.369133188854903e-05,
 7.97384709585458e-05,
 0.00016105372924357653,
 9.114169370150194e-05,
 0.0001346935605397448,
 5.309180050971918e-05]

In [47]:
test['probs'].apply(lambda x: [p for p in x])

0       [[, 0, ., 0, 0, 0, 5, 7, 8, 8, 4, 3, 1, 1, 5, ...
1       [[, 3, ., 0, 2, 2, 1, 1, 5, 1, 5, 1, 9, 0, 7, ...
2       [[, 0, ., 9, 9, 9, 8, 8, 9, 9, 6, 9, 8, 2, 5, ...
3       [[, 0, ., 4, 1, 1, 2, 0, 5, 6, 1, 9, 5, 7, 3, ...
4       [[, 0, ., 0, 0, 1, 2, 2, 4, 5, 7, 5, 5, 6, 4, ...
                              ...                        
7760    [[, 0, ., 6, 5, 5, 9, 9, 5, 0, 7, 0, 9, 3, 4, ...
7761    [[, 0, ., 9, 9, 9, 8, 8, 6, 1, 5, 5, 1, 2, 8, ...
7762    [[, 1, ., 0, 8, 2, 1, 4, 0, 5, 3, 3, 9, 8, 1, ...
7763    [[, 0, ., 9, 9, 2, 3, 4, 4, 6, 7, 7, 4, 4, 8, ...
7764    [[, 0, ., 9, 9, 9, 7, 2, 1, 2, 2, 9, 0, 7, 6, ...
Name: probs, Length: 7765, dtype: object

In [73]:
pred_probs = []
for idx, r in test.iterrows():
    zeros = np.zeros(30)
    for jdx, p in zip(label_idx, eval(r['probs'])):
        zeros[jdx] = p
    pred_probs.append(list(zeros))

In [75]:
test['probs']=pred_probs

In [94]:
test.loc[2,'pred_label']

'no_relation'

In [90]:
test

,id,gold_label,pred_label,probs,zeros
0,0,100,1,"[0, 303003]",0
1,1,100,per:alternate_names,"[0.0, 3.022115151907201e-06, 0.0, 4.5307548134...",0
2,2,100,no_relation,"[0.0, 0.9998899698257446, 0.0, 7.7153172242105...",0
3,3,100,per:other_family,"[0.0, 0.41120561957359314, 0.0, 0.000210893806...",0
4,4,100,org:top_members/employees,"[0.0, 0.0012245755642652512, 0.0, 0.9984153509...",0
...,...,...,...,...,...
7760,7760,100,no_relation,"[0.0, 0.6559950709342957, 0.0, 0.0007126444834...",0
7761,7761,100,no_relation,"[0.0, 0.999886155128479, 0.0, 4.99390716868219...",0
7762,7762,100,org:top_members/employees,"[0.0, 1.0821405339811463e-05, 0.0, 0.999811589...",0
7763,7763,100,no_relation,"[0.0, 0.9923446774482727, 0.0, 0.0001384720671...",0


In [102]:
train.iloc[[1,22,3]]

,id,sentence,subject_entity,object_entity,label,source
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
22,22,"박흥식은 첫 부인과의 사이에 장녀 박병숙을 두었고, 두 번째 부인은 경희대학교 교수...","{'word': '박흥식', 'start_idx': 0, 'end_idx': 2, ...","{'word': '한인하', 'start_idx': 57, 'end_idx': 59...",relation,wikipedia
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",relation,wikitree


In [103]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [105]:
cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [106]:
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",relation,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",relation,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",relation,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",relation,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",relation,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [111]:
def label_to_num(label):
    num_label = []
    with open('/opt/ml/code/dict_label_to_num.pkl', 'rb') as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

In [112]:
label = label_to_num(train['label'].values)

In [163]:
for idx , (train_idx , val_idx) in enumerate(cv.split(train,label)):
    # print(label)
    # train_label = np.array(label)[train_idx].tolist()
    # print(train_data)
    # print(train_label)
    train_id = train_idx
    traind_label = np.array(label)[train_idx].tolist()

In [150]:
type(val_label)

list

In [138]:
train_idx

array([    0,     1,     2, ..., 32466, 32468, 32469])

In [141]:
train.iloc[train_idx]

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '이창진', 'start_idx': 12, 'end_idx': 14...","{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...",per:title,wikitree
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [158]:
len(train_label)

25976

In [168]:
train.iloc[train_id]['label'].values

array(['no_relation', 'no_relation', 'org:member_of', ...,
       'per:colleagues', 'no_relation', 'org:top_members/employees'],
      dtype=object)

In [132]:
train_data, dev_data, train_label, dev_label = train_test_split(train, label, test_size=0.2, random_state=42, stratify=label)

In [133]:
train_data

,id,sentence,subject_entity,object_entity,label,source
10616,10616,"전현무 소속사 SM C&C는 ""'아나운서'라는 직업적 공통분모 속에서 선후배 간 좋...","{'word': '전현무', 'start_idx': 0, 'end_idx': 2, ...","{'word': 'SM C&C', 'start_idx': 8, 'end_idx': ...",per:employee_of,wikitree
20347,20347,"또한, 자유한국당 나경원 의원과 송언석 의원의 보좌관을 지내며 지역 민원과 예산을 ...","{'word': '이철우', 'start_idx': 54, 'end_idx': 56...","{'word': '자유한국당', 'start_idx': 4, 'end_idx': 8...",per:employee_of,wikitree
7979,7979,"지난 설명절 연휴기간에는 전동평 영암군수, 유나종 군의회부의장, 오정현 농협영암군지...","{'word': '영암군', 'start_idx': 91, 'end_idx': 93...","{'word': '전동평', 'start_idx': 14, 'end_idx': 16...",org:top_members/employees,wikitree
5225,5225,1932년 3월 만주국 건국 이후 동북군의 일부였던 장쉐량의 6만 명 규모의 부대 ...,"{'word': '장쉐량', 'start_idx': 29, 'end_idx': 31...","{'word': '동북군', 'start_idx': 19, 'end_idx': 21...",per:employee_of,wikipedia
24488,24488,더불어민주당 안호영 '완주·진안·무주·장수' 국회의원 예비후보가 사회서비스 분야의 ...,"{'word': '안호영', 'start_idx': 7, 'end_idx': 9, ...","{'word': '더불어민주당', 'start_idx': 0, 'end_idx': ...",per:employee_of,wikitree
...,...,...,...,...,...,...
17363,17363,‘땡큐쏘마취 휘플위크’ 기간에는 ‘휘플위크 패키지’ 평창과 제주 2가지 상품이 출시...,"{'word': '휘닉스 제주 섭지코지', 'start_idx': 57, 'end_...","{'word': '휘닉스 평창', 'start_idx': 49, 'end_idx':...",no_relation,wikitree
23494,23494,"(Vita S. Kiliani, in Canisius, Lect. Antiquæ, ...","{'word': '유스티니아누스', 'start_idx': 71, 'end_idx'...","{'word': '콘스탄티노폴리스', 'start_idx': 118, 'end_id...",no_relation,wikipedia
5201,5201,"정약용과 그의 둘째 형 정약전은 정약종과는 달리 이미 천주교를 버린 뒤였으나, 노론...","{'word': '정약용', 'start_idx': 0, 'end_idx': 2, ...","{'word': '천주교', 'start_idx': 30, 'end_idx': 32...",no_relation,wikipedia
22395,22395,경기 고양시(시장 이재준) 일산동구보건소 치매안심센터는 치매안심마을(일산동구 중산동...,"{'word': '일산동구', 'start_idx': 15, 'end_idx': 1...","{'word': '고양시', 'start_idx': 3, 'end_idx': 5, ...",org:member_of,wikitree


In [151]:
type(dev_label)

list

In [169]:
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [183]:
sub = set(train['subject_entity'].apply(lambda x: eval(x)['type']))
ob = set(train['object_entity'].apply(lambda x: eval(x)['type']))

In [197]:
rel = set(train['label'])

In [198]:
rel

{'no_relation',
 'org:alternate_names',
 'org:dissolved',
 'org:founded',
 'org:founded_by',
 'org:member_of',
 'org:members',
 'org:number_of_employees/members',
 'org:place_of_headquarters',
 'org:political/religious_affiliation',
 'org:product',
 'org:top_members/employees',
 'per:alternate_names',
 'per:children',
 'per:colleagues',
 'per:date_of_birth',
 'per:date_of_death',
 'per:employee_of',
 'per:origin',
 'per:other_family',
 'per:parents',
 'per:place_of_birth',
 'per:place_of_death',
 'per:place_of_residence',
 'per:product',
 'per:religion',
 'per:schools_attended',
 'per:siblings',
 'per:spouse',
 'per:title'}

In [217]:
a=pd.concat([train['label'],train['subject_entity'].apply(lambda x: eval(x)['type']),train['object_entity'].apply(lambda x: eval(x)['type'])],axis=1).drop_duplicates()

In [221]:
a.sort_values('label').reset_index(drop=True).to_csv('/opt/ml/code/prediction/sor.csv')

In [192]:
out=[]
for subj_type in sub:
    for obj_type in ob:
        subj_data = train[train['subject_entity'].apply(lambda x: eval(x)['type']==subj_type)]
        obj_data = subj_data[subj_data['object_entity'].apply(lambda x: eval(x)['type']==obj_type)]
        out.append([subj_type,obj_type,set(obj_data['label'])])        

In [196]:
pd.DataFrame(out,columns=['subj','obj','relation']).to_csv('/opt/ml/code/prediction/sor.csv')

In [ ]:
ID_TO_TYPE_PAIR = {
    0: 'ORG_PER',
    1: 'ORG_COUNTRY',
    2: 'PERSON_PERSON',
    3: 'PERSON_COUNTRY',
    4: 'ORGANIZATION_ORGANIZATION',
    5: 'PERSON_CITY',
    6: 'PERSON_NATIONALITY',
    7: 'PERSON_ORGANIZATION',
    8: 'ORGANIZATION_LOCATION',
    9: 'PERSON_LOCATION',
    10: 'PERSON_STATE_OR_PROVINCE',
    11: 'PERSON_DATE',
    12: 'ORGANIZATION_DATE',
}

In [224]:
train

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [225]:
train.items()

<generator object DataFrame.items at 0x7fbc63069eb0>

In [227]:
nomatch90=pd.read_csv('/opt/ml/code/prediction/no_match_90.csv')

In [230]:
pd.set_option('display.max_rows',None)

In [235]:
nomatch90[nomatch90['gold_label']==nomatch90['second_label']]

,Unnamed: 0,sentence,subj_word,subj_type,obj_word,obj_type,gold_label,pred_label,second_label,gold_prob,pred_prob,second_prob,prob_diff,prse_diff
0,1026,"1991년 개헌후 처음 벌어졌던 기초지방선거에서 참패한 후, 9월 10일 합당에 합...",신민주연합,ORG,민주당,ORG,no_relation,org:alternate_names,no_relation,0.3459,0.5342,0.3459,0.1883,0.1883
1,5677,"오르내림 (OLNL)이 소속된 크루는 쥬스오버알콜 (juiceoveralchol),...",OLNL,ORG,WYBH,ORG,no_relation,org:alternate_names,no_relation,0.3685,0.6259,0.3685,0.2574,0.2574
2,5782,"약칭은 한민당. 1945년 9월 16일에 조선민족당, 한국국민당 등이 만든 정당으로...",한민당,ORG,"조선민족당, 한국국민당",ORG,no_relation,org:alternate_names,no_relation,0.4267,0.5687,0.4267,0.1420,0.1420
6,708,이와 같이 2019년 3월 1일 자유조선은 조선민주주의인민공화국의 김정은 체제의 북...,자유조선,ORG,2019년 3월 1일,DAT,no_relation,org:founded,no_relation,0.3070,0.6832,0.3070,0.3763,0.3763
7,1664,법률요건을 충족시키고 원로원에 특권을 불하받은 핀레이슨은 1820년 탐페레에 소면기...,핀레이슨,PER,1820년,DAT,no_relation,org:founded,no_relation,0.2124,0.7808,0.2124,0.5684,0.5684
8,2095,1988년 3월 6일에 완공하여 완공 당시 그린 스타디움 고베라는 애칭을 사용하여 ...,그린 스타디움 고베,ORG,1988년 3월 6일,DAT,no_relation,org:founded,no_relation,0.1595,0.8316,0.1595,0.6720,0.6720
9,2755,1947년 11월 1일 정부로부터 학교인가를 받아 같은해 11월 3일 단국대학교를 ...,단국대학교사범대학부속중학교,ORG,1947년,DAT,no_relation,org:founded,no_relation,0.0939,0.8896,0.0939,0.7957,0.7957
10,3676,"1995년 정계에 복귀한 김대중의 신당 새정치국민회의의 발기인으로 참여했고, 이듬해...",새정치국민회의,ORG,1996년,DAT,no_relation,org:founded,no_relation,0.1700,0.8057,0.1700,0.6357,0.6357
11,806,양평군은 도농복합 도시로 다양한 토지이용현황이 분포되어 중앙정부 및 한국국토정보공사...,한국국토정보공사,ORG,국토교통부,ORG,no_relation,org:member_of,no_relation,0.4567,0.5406,0.4567,0.0838,0.0838
12,1075,페트릭 피어즈가 이끄는 아일랜드 의용군과 제임스 코놀리가 이끄는 아일랜드 시민군은 ...,아일랜드 의용군,ORG,아일랜드 시민군,ORG,no_relation,org:member_of,no_relation,0.2596,0.7329,0.2596,0.4733,0.4733
